In [1]:
#We first define the message and reduce function as usual. Since the aggregation 
#on a node u only involves summing over the neighbors’ representations hv, 
#we can simply use builtin functions:
import dgl
import dgl.function as fn
import torch as th
import torch.nn as nn
import torch.nn.functional as F
from dgl import DGLGraph

gcn_msg = fn.copy_src(src='h', out='m')
gcn_reduce = fn.sum(msg='m', out='h')

Using backend: pytorch


In [2]:
#We then proceed to define the GCNLayer module. 
#A GCNLayer essentially performs message passing on all 
#the nodes then applies a fully-connected layer.

class GCNLayer(nn.Module):
    def __init__(self, in_feats, out_feats):
        super(GCNLayer, self).__init__()
        self.linear = nn.Linear(in_feats, out_feats)

    def forward(self, g, feature):
        # Creating a local scope so that all the stored ndata and edata
        # (such as the `'h'` ndata below) are automatically popped out
        # when the scope exits.
        with g.local_scope():
            g.ndata['h'] = feature
            g.update_all(gcn_msg, gcn_reduce)
            h = g.ndata['h']
            return self.linear(h)

#The forward function is essentially the same as any other commonly seen NNs model in PyTorch. 
#We can initialize GCN like any nn.Module. For example, let’s define a simple neural network consisting 
#of two GCN layers. Suppose we are training the classifier for the cora dataset (the input feature size 
#is 1433 and the number of classes is 7). 
#The last GCN layer computes node embeddings, so the last layer in general does not apply activation.

In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.layer1 = GCNLayer(1433, 16)
        self.layer2 = GCNLayer(16, 7)

    def forward(self, g, features):
        x = F.relu(self.layer1(g, features))
        x = self.layer2(g, x)
        return x
net = Net()
print(net)

Net(
  (layer1): GCNLayer(
    (linear): Linear(in_features=1433, out_features=16, bias=True)
  )
  (layer2): GCNLayer(
    (linear): Linear(in_features=16, out_features=7, bias=True)
  )
)


In [4]:
#We load the cora dataset using DGL’s built-in data module

from dgl.data import citation_graph as citegrh
import networkx as nx
def load_cora_data():
    data = citegrh.load_cora()
    features = th.FloatTensor(data.features)
    labels = th.LongTensor(data.labels)
    train_mask = th.BoolTensor(data.train_mask)
    test_mask = th.BoolTensor(data.test_mask)
    g = DGLGraph(data.graph)
    return g, features, labels, train_mask, test_mask

In [5]:
#When a model is trained, we can use the following method to evaluate 
#zGCthe performance of the model on the test dataset:

def evaluate(model, g, features, labels, mask):
    model.eval()
    with th.no_grad():
        logits = model(g, features)
        logits = logits[mask]
        labels = labels[mask]
        _, indices = th.max(logits, dim=1)
        correct = th.sum(indices == labels)
        return correct.item() * 1.0 / len(labels)

In [6]:
#We then train the model as follows:

import time
import numpy as np
g, features, labels, train_mask, test_mask = load_cora_data()
optimizer = th.optim.Adam(net.parameters(), lr=1e-2)
dur = []
for epoch in range(50):
    if epoch >=3:
        t0 = time.time()

    net.train()
    logits = net(g, features)
    logp = F.log_softmax(logits, 1)
    loss = F.nll_loss(logp[train_mask], labels[train_mask])

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch >=3:
        dur.append(time.time() - t0)

    acc = evaluate(net, g, features, labels, test_mask)
    print("Epoch {:05d} | Loss {:.4f} | Test Acc {:.4f} | Time(s) {:.4f}".format(
            epoch, loss.item(), acc, np.mean(dur)))

Loading from cache failed, re-processing.
Finished data loading and preprocessing.
  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done saving data into cached files.
Epoch 00000 | Loss 1.9650 | Test Acc 0.1750 | Time(s) nan
Epoch 00001 | Loss 1.8394 | Test Acc 0.2220 | Time(s) nan


C:\Users\rothg\Anaconda3\lib\site-packages\dgl\data\utils.py:285: UserWarning: Property dataset.feat will be deprecated, please use g.ndata['feat'] instead.
  warnings.warn('Property {} will be deprecated, please use {} instead.'.format(old, new))
C:\Users\rothg\Anaconda3\lib\site-packages\dgl\data\utils.py:285: UserWarning: Property dataset.label will be deprecated, please use g.ndata['label'] instead.
  warnings.warn('Property {} will be deprecated, please use {} instead.'.format(old, new))
C:\Users\rothg\Anaconda3\lib\site-packages\dgl\data\utils.py:285: UserWarning: Property dataset.train_mask will be deprecated, please use g.ndata['train_mask'] instead.
  warnings.warn('Property {} will be deprecated, please use {} instead.'.format(old, new))
C:\Users\rothg\Anaconda3\lib\site-packages\dgl\data\utils.py:285: UserWarning: Property dataset.test_mask will be deprecated, please use g.ndata['test_mask'] instead.
  warnings.warn('Property {} will be deprecated, please use {} instead.'.fo

Epoch 00002 | Loss 1.7255 | Test Acc 0.3640 | Time(s) nan
Epoch 00003 | Loss 1.6197 | Test Acc 0.4270 | Time(s) 0.0209
Epoch 00004 | Loss 1.5261 | Test Acc 0.4950 | Time(s) 0.0209
Epoch 00005 | Loss 1.4465 | Test Acc 0.5540 | Time(s) 0.0199
Epoch 00006 | Loss 1.3763 | Test Acc 0.5910 | Time(s) 0.0194
Epoch 00007 | Loss 1.3092 | Test Acc 0.6160 | Time(s) 0.0191
Epoch 00008 | Loss 1.2420 | Test Acc 0.6460 | Time(s) 0.0193
Epoch 00009 | Loss 1.1749 | Test Acc 0.6640 | Time(s) 0.0191
Epoch 00010 | Loss 1.1082 | Test Acc 0.6830 | Time(s) 0.0188
Epoch 00011 | Loss 1.0424 | Test Acc 0.7030 | Time(s) 0.0186
Epoch 00012 | Loss 0.9795 | Test Acc 0.7120 | Time(s) 0.0186
Epoch 00013 | Loss 0.9205 | Test Acc 0.7050 | Time(s) 0.0185
Epoch 00014 | Loss 0.8649 | Test Acc 0.7070 | Time(s) 0.0187
Epoch 00015 | Loss 0.8117 | Test Acc 0.7100 | Time(s) 0.0186
Epoch 00016 | Loss 0.7602 | Test Acc 0.7160 | Time(s) 0.0185
Epoch 00017 | Loss 0.7112 | Test Acc 0.7250 | Time(s) 0.0184
Epoch 00018 | Loss 0.6653 |